In [1]:
#Better
!pip install requests BeautifulSoup4 fire

In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import os, sys

import fire

In [3]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [4]:
response = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa',tag='h2')
response

['100. Jeffrey Gettleman (@gettleman)',
 '99. Africa24 Media (@a24media)',
 '98. Scapegoat (@andiMakinana)',
 '97. Africa Check (@AfricaCheck)',
 '96. James Copnall (@JamesCopnall)',
 '95. Online Africa (@oafrica)',
 '94. Patrick Ngowi (@PatrickNgowi)',
 '93. DOS African Affairs (@StateAfrica)',
 '92. MoadowAJE (@Moadow)',
 '91. Brendan Boyle (@BrendanSAfrica)',
 '90. City of Tshwane (@CityTshwane)',
 '89. VISI Magazine (@VISI_Mag)',
 '88. andBeyond (@andBeyondSafari)',
 '87. This Is Africa (@ThisIsAfricaTIA)',
 '86. Sarah Carter (@sarzss)',
 '85. The EIU Africa team (@TheEIU_Africa)',
 '84. Investing In Africa (@InvestInAfrica)',
 '83. Barry Malone (@malonebarry)',
 '82. ARTsouthAFRICA (@artsouthafrica)',
 '81. Kahn Morbee (@KahnMorbee)',
 '80. Jamal Osman (@JamalMOsman)',
 '79. iamsuede™ (@iamsuede)',
 '78. Mike Stopforth (@mikestopforth)',
 '77. Equal Education (@equal_education)',
 '76. Tristan McConnell (@t_mcconnell)',
 '75. Kate Forbes (@forbeesta)',
 '74. Vanessa Raphaely (@hur

In [5]:
list = []
for rows in response:
    res = re.findall(r'\(.*?\)', rows)
    list.append(res)

influencers_tweet_handles = []
for l in list:
    for rows in l:
        if rows.startswith('(@'):
            influencers_tweet_handles.append(rows.strip('(,)'))
            
influencers_tweet_handles

['@gettleman',
 '@a24media',
 '@andiMakinana',
 '@AfricaCheck',
 '@JamesCopnall',
 '@oafrica',
 '@PatrickNgowi',
 '@StateAfrica',
 '@Moadow',
 '@BrendanSAfrica',
 '@CityTshwane',
 '@VISI_Mag',
 '@andBeyondSafari',
 '@ThisIsAfricaTIA',
 '@sarzss',
 '@TheEIU_Africa',
 '@InvestInAfrica',
 '@malonebarry',
 '@artsouthafrica',
 '@KahnMorbee',
 '@JamalMOsman',
 '@iamsuede',
 '@mikestopforth',
 '@equal_education',
 '@t_mcconnell',
 '@forbeesta',
 '@hurricanevaness',
 '@BBCKarenAllen',
 '@jaxpanik',
 '@thisisafrica',
 '@audisouthafrica',
 '@ONEinAfrica',
 '@Hamza_Africa',
 '@drewfhinshaw',
 '@africatechie',
 '@cx73',
 '@ayittey',
 '@MercedesBenz_SA',
 '@africagathering',
 '@okayafrica',
 '@mary_harper',
 '@savetherhino',
 '@africamedia_CPJ',
 '@TechCentral',
 '@GautengProvince',
 '@Aynte',
 '@daniel_howden',
 '@rangerdiaries',
 '@TheStar_news',
 '@schneiderhome',
 '@Afrinnovator',
 '@theafricareport',
 '@CityofJoburgZA',
 '@ThinkAfricaFeed',
 '@AfricaGoodNews',
 '@willintune',
 '@cnbcafrica',
 

In [6]:
url = 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter'
response = simple_get(url)

res_cov = get_elements(response, tag='div', search={})
res_cov

['Issues',
 'Regions',
 'Insights & impact',
 'People',
 'Programs',
 'Events',
 '',
 'Get involved',
 'Support the Council',
 'Sign up',
 'About the Council',
 'Media',
 'Careers',
 'Home',
 'Issues',
 'Regions',
 'Search',
 'Menu',
 '+',
 'Issues',
 'Regions',
 'Insights & impact',
 'People',
 'Programs',
 'Events',
 '',
 'Get involved',
 'Support the Council',
 'Sign up',
 'About the Council',
 'Media',
 'Careers',
 '',
 'Search Suggestions',
 '{{ suggestion }}',
 '',
 'Total Results {{ currentCount }}/{{ totalCount }}',
 '{{ searchResult.post_type }}',
 '{{ searchResult.date }}',
 '{{ searchResult.author.name }}',
 'Load More',
 'Issues',
 'Region',
 'Focus',
 'Filter Results',
 'Hide',
 'Tue, Mar 24, 2020',
 'African leaders respond to coronavirus… on Twitter',
 'AfricaSource',
 'by',
 'Luke Tyburski',
 'Africa',
 'Coronavirus',
 'Media',
 'Resilience & Society',
 'Ivorian President Alassane Ouattara shows off proper sanitation practices to his Twitter following. (Twitter/@AOuatta

In [7]:
list = []
for tweets in res_cov:
    res = re.findall(r'\(.*?\)', tweets)
    list.append(res)

tweet_handles = []
for l in list:
    for tweets in l:
        if tweets.startswith('(@'):
            tweet_handles.append(tweets.strip('(,)'))
            
tweet_handles


['@EswatiniGovern1',
 '@MalawiGovt',
 '@hagegeingob',
 '@FinanceSC',
 '@PresidencyZA',
 '@mohzambia',
 '@edmnangagwa',
 '@MinSantedj',
 '@hawelti',
 '@StateHouseKenya',
 '@PaulKagame',
 '@M_Farmaajo',
 '@SouthSudanGov',
 '@SudanPMHamdok',
 '@TZSpokesperson',
 '@KagutaMuseveni',
 '@angola_Mirex',
 '@willynyamitwe',
 '@Cherif_MZ',
 '@Presidence_RDC',
 '@PresidentABO',
 '@PresidenceBenin',
 '@rochkaborepf',
 '@PresidenciaCV',
 '@AOuattara_PRCI',
 '@Presidency_GMB',
 '@NAkufoAddo',
 '@President_GN',
 '@USEmbalo',
 '@PresidenceMali',
 '@CheikhGhazouani',
 '@IssoufouMhm',
 '@MBuhari',
 '@Macky_Sall',
 '@PresidentBio',
 '@MSPS_Togo',
 '@EswatiniGovern1',
 '@MalawiGovt',
 '@hagegeingob',
 '@FinanceSC',
 '@PresidencyZA',
 '@mohzambia',
 '@edmnangagwa',
 '@MinSantedj',
 '@hawelti',
 '@StateHouseKenya',
 '@PaulKagame',
 '@M_Farmaajo',
 '@SouthSudanGov',
 '@SudanPMHamdok',
 '@TZSpokesperson',
 '@KagutaMuseveni',
 '@angola_Mirex',
 '@willynyamitwe',
 '@Cherif_MZ',
 '@Presidence_RDC',
 '@PresidentAB

In [8]:
import tweepy 

#Variables that contains the user credentials to access Twitter API 
consumer_key = os.environ.get('TWITTER_API_KEY')
consumer_secret = os.environ.get('TWITTER_API_SECRET')
access_token = os.environ.get('TWITTER_ACCESS_TOKEN')
access_token_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')

In [9]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [10]:
lst = []
for handle in tweet_handles:
  user = api.get_user(handle)
  lst.append([user.name, user.followers_count])
lst
  


[['Eswatini Government', 11236],
 ['Malawi Government', 39056],
 ['Hage G. Geingob', 191847],
 ['Seychelles Ministry of Finance', 126],
 ['Presidency | South Africa 🇿🇦', 1595884],
 ['Ministry of Health Zambia', 7099],
 ['President of Zimbabwe', 544663],
 ['MinSantédj', 2919],
 ['Yemane G. Meskel', 65918],
 ['State House Kenya', 1101140],
 ['Paul Kagame', 1979334],
 ['Mohamed Farmaajo', 421671],
 ['South Sudan Government', 2525],
 ['Abdalla Hamdok', 368551],
 ['TanzaniaSpokesperson', 3998],
 ['Yoweri K Museveni', 1807308],
 ['MOFA/MRE -(Angola)', 3260],
 ['Amb. Willy Nyamitwe', 107189],
 ['Chérif Mahamat Zene', 18290],
 ['Présidence RDC 🇨🇩', 338709],
 ['Ali Bongo Ondimba', 172891],
 ['Présidence du Bénin', 30030],
 ['Roch KABORE', 255373],
 ['Presidente Cabo Verde', 3267],
 ['Alassane Ouattara', 828389],
 ['State House of The Gambia', 10654],
 ['Nana Akufo-Addo', 1502177],
 ['Alpha CONDÉ', 706],
 ['Umaro Sissoco Embalo', 8462],
 ['Presidence Mali', 228161],
 ['Mohamed Cheikh El Ghazouan

In [11]:
leaders_handle = pd.DataFrame(lst, columns=['leader_handle', 'followers_count'])
leaders_handle 

,leader_handle,followers_count
0,Eswatini Government,11236
1,Malawi Government,39056
2,Hage G. Geingob,191847
3,Seychelles Ministry of Finance,126
4,Presidency | South Africa 🇿🇦,1595884
...,...,...
211,Issoufou Mahamadou,115737
212,Muhammadu Buhari,3265877
213,Macky Sall,1371252
214,President Julius Maada Bio,14633


In [19]:
sort_leader_rank = leaders_handle.sort_values(by='followers_count', ascending=False)
sort_leader_rank = sort_leader_rank.drop_duplicates(subset='leader_handle')
top10_leaders = sort_leader_rank.iloc[:9]
top10_leaders

,leader_handle,followers_count
212,Muhammadu Buhari,3265877
190,Paul Kagame,1979339
159,Yoweri K Museveni,1807313
184,Presidency | South Africa 🇿🇦,1595889
98,Nana Akufo-Addo,1502180
213,Macky Sall,1371252
189,State House Kenya,1101142
168,Alassane Ouattara,828391
186,President of Zimbabwe,544664


In [20]:
top10_leaders_handles = top10_leaders['leader_handle']
top10_leaders_handles

212                Muhammadu Buhari
190                     Paul Kagame
159               Yoweri K Museveni
184    Presidency | South Africa 🇿🇦
98                  Nana Akufo-Addo
213                      Macky Sall
189               State House Kenya
168               Alassane Ouattara
186           President of Zimbabwe
Name: leader_handle, dtype: object

Top Africa  influencers' followers count 

In [14]:
lst_top = []
for results in influencers_tweet_handles:
  try:
    user = api.get_user(results)
    lst_top.append([user.name, user.followers_count])
  except Exception:
    pass
lst_top

[['Jeffrey Gettleman', 25702],
 ['A24 Media', 31293],
 ['Scapegoat', 101181],
 ['Africa Check', 68034],
 ['James Copnall', 21960],
 ['Online Africa', 31301],
 ['Patrick Ngowi', 22796],
 ['SEO Expert Vancouver', 8],
 ['Mohammed Adow', 50956],
 ['Brendan Boyle', 28462],
 ['City of Tshwane', 438575],
 ['VISI Magazine', 52898],
 ['This Is Africa', 46774],
 ['Sarah Carter', 11928],
 ['The EIU Africa team', 24859],
 ['Investing In Africa', 49682],
 ['Barry Malone', 28423],
 ['ArtSouthAfrica', 116],
 ['Kahn Morbee', 17442],
 ['Jamal Osman', 93913],
 ['iamsuede™', 18240],
 ['Mike Stopforth', 21349],
 ['Equal Education', 62156],
 ['Tristan McConnell', 18505],
 ['Kate Forbes', 20000],
 ['Jax Panik', 7489],
 ['T.I.A', 6],
 ['audisouthafrica', 215146],
 ['ONE in Africa', 232861],
 ['Hamza Mohamed', 98710],
 ['Rebecca Enonchong', 105905],
 ['marais', 70792],
 ['George Ayittey', 17073],
 ['Mercedes-Benz SA', 249753],
 ['Africa Gathering', 48951],
 ['OkayAfrica', 105211],
 ['Mary Harper', 58210],
 ['

In [15]:
influencers_handle = pd.DataFrame(lst_top, columns=['influencer_handle', 'followers_count'])
influencers_handle

,influencer_handle,followers_count
0,Jeffrey Gettleman,25702
1,A24 Media,31293
2,Scapegoat,101181
3,Africa Check,68034
4,James Copnall,21960
...,...,...
88,Julius Sello Malema,3123220
89,News24,3572984
90,jdwtweet,18
91,Gareth Cliff,1974288


In [21]:
sort_influencer_rank = influencers_handle.sort_values(by='followers_count', ascending=False)
top10_African_influencer = sort_influencer_rank.iloc[:9]
top10_African_influencer

,influencer_handle,followers_count
92,Trevor Noah,10798854
89,News24,3572984
88,Julius Sello Malema,3123220
91,Gareth Cliff,1974288
79,Euphonik™♛,1753248
87,Helen Zille,1443080
82,MTV Base Africa,1415204
85,5FM,1164028
81,Cricket South Africa,1159582


In [22]:
top10_influencer_handles = top10_African_influencer['influencer_handle']
top10_influencer_handles

92             Trevor Noah
89                  News24
88     Julius Sello Malema
91            Gareth Cliff
79              Euphonik™♛
87             Helen Zille
82         MTV Base Africa
85                     5FM
81    Cricket South Africa
Name: influencer_handle, dtype: object

In [23]:
top10_influencer_handles.to_csv('top10_influencers.csv', index=False)
top10_leaders_handles.to_csv('top10_leaders.csv', index=False)